In [1]:
import psycopg2
import pandas as pd

myCon = psycopg2.connect(
    dbname='ecommerce_db',
    user='postgres',
    password='riska06',
    host='localhost',
)


In [2]:
cur = myCon.cursor()
cur.execute("SELECT * FROM customers;")
rows = cur.fetchall()

for d in rows:
    print(d)
    
cur.close()

(1, 25, Decimal('500.50'), 5)
(2, 30, Decimal('1500.75'), 15)
(3, 22, Decimal('300.10'), 3)
(4, 35, Decimal('1200.00'), 12)
(5, 28, Decimal('800.60'), 8)


In [3]:
query = """

SELECT 
    t.customer_id, 
    t.transactions_date, 
    t.product_category, 
    t.amount, 
    c.age  -- Include only age as a customer attribute
FROM 
    transactions t
JOIN 
    customers c ON t.customer_id = c.customer_id
ORDER BY 
    t.customer_id, t.transactions_date;

"""


In [4]:
df = pd.read_sql(query, myCon)

myCon.close()

print(df.head())

   customer_id transactions_date product_category  amount  age
0            1        2023-01-10      Electronics   150.0   25
1            1        2023-01-15          Fashion    80.0   25
2            2        2023-01-12            Books    20.0   30
3            2        2023-01-18      Electronics   200.0   30
4            3        2023-01-14            Books    15.0   22


C:\Users\Asus\AppData\Local\Temp\ipykernel_7956\3710778092.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, myCon)


In [5]:
# Pastikan transaction_date dalam format datetime
df['transactions_date'] = pd.to_datetime(df['transactions_date'])

# RECENCY: Berapa hari sejak transaksi terakhir
recency_df = df.groupby('customer_id')['transactions_date'].max()
recency_df = pd.DataFrame({'recency': (df['transactions_date'].max() - recency_df).dt.days})

# FREQUENCY: Berapa kali transaksi
frequency_df = df.groupby('customer_id')['transactions_date'].count()
frequency_df = pd.DataFrame({'frequency': frequency_df})

# MONETARY: Total uang yang dibelanjakan
monetary_df = df.groupby('customer_id')['amount'].sum()
monetary_df = pd.DataFrame({'monetary': monetary_df})

In [7]:
# Gabungkan recency, frequency, dan monetary
customer_data = recency_df.merge(frequency_df, on='customer_id').merge(monetary_df, on='customer_id')

# Tambahkan atribut pelanggan (seperti umur)
customer_attributes = df[['customer_id', 'age']].drop_duplicates()
customer_data = customer_data.merge(customer_attributes, on='customer_id')

# Tampilkan hasil
print(customer_data.head())

   customer_id  recency  frequency  monetary  age
0            1        5          2     230.0   25
1            2        2          2     220.0   30
2            3        6          1      15.0   22
3            4        4          1      50.0   35
4            5        0          1     300.0   28
